In [1]:
import os

repo_dir = os.environ.get("REPO_DIR")
code_dir = os.path.join(repo_dir, "code/")
data_dir = os.path.join(repo_dir, "data/")

os.chdir(code_dir)

import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import pickle
import sklearn 
import sys
import pandas as pd
from importlib import reload

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
import seaborn as sns

from scipy.stats import spearmanr, mode

import geopandas as gpd
import rasterio
import zarr

import gc

import warnings

from mosaiks.utils.imports import *

from mosaiks.utils.io import weighted_groupby
from affine import Affine

import cartopy.crs as ccrs
# Key prediction functions are here
from analysis.prediction_utils import (flatten_raster,rasterize_df)

# Predicting grid level HDI

In this notebook, we recenter HDI at the grid level for a time series from 2012 to 2021. We use an intermediate out from the notebook `hdi_preds_at_grid_time_series.ipynb`.

In [2]:
task = "Sub-national HDI"

In [3]:
data = pd.read_pickle(data_dir + "preds/raw_hdi_preds_at_grid_with_hsdl.p").reset_index()

In [4]:
# Drop preds that do not have 0 population in the HSDL dataset
data = data[data["pop_binary"] == 1].copy()

In [5]:
# Assign smallest positive pop density weight to remaining locations where pop density weights were NaN
data.loc[data["population"].isnull(), "population"] = data["population"].min()

In [6]:
data.drop(columns =["lon01","lat01"], inplace=True)
data.head()

,index,lon,lat,GDLCODE,iso_code,constant,nl,population,raw_pred_hdi,raw_pred_hdi_not_clipped,Sub-national HDI,pop_binary
1,1,43.435,-22.905,MDGr117,MDG,1,0.0,37.789421,-0.001531,-0.001531,0.408,1
2,2,43.435,-22.895,MDGr117,MDG,1,0.0,37.789421,0.029665,0.029665,0.408,1
3,3,43.435,-22.885,MDGr117,MDG,1,0.0,37.789425,0.028874,0.028874,0.408,1
4,4,43.435,-22.875,MDGr117,MDG,1,0.0,37.789421,0.021557,0.021557,0.408,1
94,94,43.435,-21.975,MDGr117,MDG,1,0.0,8.582693,0.050563,0.050563,0.408,1


In [7]:
data = data.drop(columns = task)

## Merge with time series of GDL data

In [8]:
t_df = pd.read_csv(data_dir + "raw/GDL_HDI/SHDI-SGDI-Total 7.0.csv",low_memory=False)
t_df = t_df.rename(columns = {"shdi":task})

## Re-center preds on the known ADM1 Value

In [9]:
grouped = io.weighted_groupby(data, "GDLCODE", weights_col_name="population", cols_to_agg=["raw_pred_hdi"] )
grouped.rename(columns = {"raw_pred_hdi":"weighted_avg_raw"}, inplace=True)

In [10]:
data = data.merge(grouped, left_on="GDLCODE", right_index=True)

In [11]:
data["lat10"] = np.round(np.round(data["lat"] + .05,1) - .05,2)
data["lon10"] = np.round(np.round(data["lon"] + .05,1) - .05,2)

In [12]:
year_range = np.arange(2012,2022)

for year in year_range:
    print(year)
    
    t_df_year = t_df[t_df["year"] == year]
    t_df_year = t_df_year.set_index("GDLCODE")
    
    data_year = data.merge(t_df_year[[task]], "left", left_on="GDLCODE",right_index=True)
    
    data_year["Sub-national HDI"] = data_year["Sub-national HDI"].astype(float)
    data_year["adj_factor"] = data_year["Sub-national HDI"] - data_year["weighted_avg_raw"]
    data_year["centered_pred"] = data_year["raw_pred_hdi"] + data_year["adj_factor"]
    
    
    ## Rasterize and upsample
    pre_raster = data_year.groupby(["lon10","lat10"])[["centered_pred","population","Sub-national HDI","GDLCODE"]].agg(
    {
    "population": np.nansum, # Sum the weights
    "Sub-national HDI": lambda x: mode(x, nan_policy="omit")[0], # For this col, keep the modal HDI
     "GDLCODE": lambda x: mode(x,nan_policy="omit")[0], # For this col, keep the modal parent ADM1 code
    }) #ignore NaNs for all
    
    
    #### Now for HDI we want to take the weighted average of the cells, 
    # using the same GPW pop density weights that we have been using throughout
    pre_raster = pd.concat( [pre_raster,weighted_groupby(data_year, 
                                                       ["lon10","lat10"], 
                                                       "population", 
                                                       cols_to_agg = ["centered_pred"]
                                                      )
                           ],axis=1).reset_index()
    
    pre_raster["clipped"] = np.clip(pre_raster["centered_pred"],0,1)
    
    pre_raster = pre_raster.reset_index()
    
#     pre_raster.to_pickle(data_dir + "preds/time_series/"
#            f"hdi_grid_predictions_flat_file_{year}.p")

    raster, extent = rasterize_df(pre_raster, 
                              data_colname = "clipped", 
                              grid_delta=.1, 
                              lon_col="lon10", 
                              lat_col="lat10",
                             custom_extent = (-180,180,-56,74)
                             )
    
    ####  Write grid data product as a raster
    
    meta = {'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': np.nan,
 'width': 3600,
 'height': 1300,
 'count': 1,
'crs': "EPSG:4326",
'transform': Affine(0.1, 0.0, extent[0],
        0.0, -0.1, extent[3])
       }

    raster_outpath = (data_dir + "preds/time_series"
               f"hdi_raster_predictions_{year}.tif")

    with rasterio.open(raster_outpath , "w", **meta) as dest:
         dest.write(np.array([raster]))


2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
